In [7]:
import ee
import geopandas as gpd
import geemap
import xarray as xr
import wxee

In [8]:
ee.Authenticate()
ee.Initialize(project='ee-nunezrimedio-tesina',opt_url='https://earthengine-highvolume.googleapis.com')

In [11]:

## 
def get_buenos_aires_feature():
    # Load the FAO GAUL dataset
    provinces = ee.FeatureCollection("FAO/GAUL/2015/level0")

    # Filter for Buenos Aires province
    buenos_aires_feature = provinces.filter(ee.Filter.And(
        ee.Filter.eq('ADM0_NAME', 'Argentina'),
    )).first()

    return buenos_aires_feature

# Get the Buenos Aires feature
buenos_aires_feature = get_buenos_aires_feature()
Argentina_geometry = buenos_aires_feature.geometry()

fishnet = geemap.fishnet(
    Argentina_geometry,
    rows=40,
    cols=30
)

# Get the Dynamic World image collection
#dw = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")

# Filter the collection for your desired time range
#dw_filtered = dw.filterDate('2020-01-01', '2020-12-31').filterBounds(fishnet.first().geometry())
#dw_filtered
 
dw_filtered = wxee.TimeSeries("GOOGLE/DYNAMICWORLD/V1").filterDate("2020", "2021").filterBounds(fishnet.first().geometry()).select("label")
monthly_ts = dw_filtered.aggregate_time("month", ee.Reducer.mode())

# Create a mosaic image
#image = dw_filtered.select("label").mode().clip(Argentina_geometry)
monthly_ts

In [13]:
ds = monthly_ts.wx.to_xarray(region=fishnet.first().geometry(), scale=30)

Requesting data:   0%|          | 0/11 [00:00<?, ?it/s]

In [17]:
ds.to_netcdf(r"C:\Users\nunez\Facultad\Tesina\Tesina\x_array.nc")

In [20]:
ds.describe()

AttributeError: 'Dataset' object has no attribute 'describe'

In [19]:
xr.open_dataset(r"C:\Users\nunez\Facultad\Tesina\Tesina\x_array.nc").describe()

AttributeError: 'Dataset' object has no attribute 'describe'